#### Dependencies

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import time
import multiprocessing as mp
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import sys
from contextlib import closing
import psutil
import tracemalloc
import threading
import gc
from guppy import hpy

from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
)

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


#### Global Constants

In [2]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = int(os.getenv('CPU_THREADS'))
BATCH_SIZE = 20
FEATURES_OUTPUT_FILE = "data/features.csv"

#### Data

In [3]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [4]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [5]:
def mp3_to_spectrogram(audio_path, sample_rate, create_image=False):
    mp3, _ = librosa.load(audio_path, sr=sample_rate)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db

In [6]:
spectr = mp3_to_spectrogram(DOWNLOAD_FOLDER+'/0^LlWGt_84jpg^Special Breed.mp3', 16000)

/home/alean/stuff/popcast-ai/.venv/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [7]:
print(spectr.shape)
sys.getsizeof(spectr)

(128, 10335)


5291648

In [8]:
print(spectr.shape)
sys.getsizeof(spectr)

(128, 10335)


5291648

In [9]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)

    return mel_band_energies

In [10]:
def run_essentia_algorithms(audio44k, audio16k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio16k))(audio16k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio16k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio16k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}

    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio16k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)

    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
    }

    return features

In [11]:
def extract_audio_features(audio_file):
    # Load the audio file
    audio44k = MonoLoader(filename=audio_file)()
    audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()

    # Run algorithms
    algorithm_features = run_essentia_algorithms(audio44k, audio16k)

    # Merge results
    return algorithm_features

#### Util Functions

In [12]:
def get_total_memory_usage(process):
    memory_summary = {f'Process {process.pid}': process.memory_info().rss / (1024 * 1024)}
    for child in process.children(recursive=True):
        memory_summary = memory_summary | {f'Child Process {child.pid}': child.memory_info().rss / (1024 * 1024)}
    return memory_summary

def print_memory_usage(process):
    print(get_total_memory_usage(process))
    snapshot = tracemalloc.take_snapshot()
    print(f"Top Consumer of Process {process.pid}: {snapshot.statistics('lineno')[0]}")

def monitor_memory_usage(process, kill_thread, interval=120):
    while True:
        try:
            if kill_thread.value:
                print("MONITOR THREAD KILLED")
                return
            print_memory_usage(process)
        except Exception as e:
            print(f"Thread ERROR: {e}")
            return
        time.sleep(interval)

#### Main Code

In [13]:
# Class constructed from song path
# Song path must follow this format: /some/path/(int)^(video id)^(title).mp3
#                               e.g  /some/path/0^LlWGt_84jpg^Special Breed.mp3
class SongPath:
    def __init__(self, song_path: str):
        self.path = song_path
        self.filename = os.path.basename(song_path)

        song_filename_split = self.filename.split('^')
        if len(song_filename_split) != 3:
            raise Exception("The song's filename doesn't follow the correct format: /some/path/(int)^(video id)^(title).mp3")
        
        self.index, self.video_id, self.title_with_extension = song_filename_split

        self.index = int(self.index)
        self.title = os.path.splitext(self.title_with_extension)[0]

    def __str__(self):
        return f"Idx: {self.index},  videoID: {self.video_id}, title: {self.title_with_extension}"

In [14]:
def process_song(args):
    song_path, output_file = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    #print_memory_usage(psutil.Process(os.getpid()))
    #song_results.append((song.index, song_features))

    # If file doesnt exist, create file and add headers
    print(song_features.keys())
    if not os.path.exists(output_file):
        empty_df = pd.DataFrame(columns=song_features.keys())
        empty_df.to_csv(output_file, index=False)
    with open(output_file, 'a') as f:
        pd.DataFrame([song_features]).to_csv(f, header=False, index=False)

def process_song(args):
    song_path, song_results = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    #print_memory_usage(psutil.Process(os.getpid()))
    song_results.append((song.index, song_features))

In [15]:
def process_songs():                   
    tracemalloc.start()

    # Get downloaded songs paths
    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])
    #songs_data_lower, songs_data_higher = [0, len(song_paths)//6]
    songs_data_lower, songs_data_higher = [0, 12]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    # Monitor processes' memory for debugging
    main_process = psutil.Process(os.getpid())
    print(f"Main Process ID: {main_process.pid}")
    memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process))
    memory_thread.daemon = True  # Ensure the thread will exit when the main program exits
    memory_thread.start()

    # Determine output file (needs to be a brand new file)
    output_file_name, output_file_ext = os.path.splitext(FEATURES_OUTPUT_FILE)
    output_file_suffix = 1
    while os.path.exists(output_file_name + str(output_file_suffix) + output_file_ext):
        output_file_suffix += 1
    output_file = output_file_name + str(output_file_suffix) + output_file_ext

    with mp.Pool(processes=CPU_THREADS, maxtasksperchild=5) as pool:
        with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
            args = [(song_path, output_file) for song_path in song_paths]
            for _ in pool.imap(process_song, args, chunksize=1):
                pbar.update(1)

In [16]:
def process_songs():                   
    tracemalloc.start()

    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])

    songs_data_lower, songs_data_higher = [0, len(song_paths)//6]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    with mp.Manager() as manager:
        kill_thread = manager.Value('b', False)
        shared_song_results = manager.list()
        main_process = psutil.Process(os.getpid())
        print(f"Main Process ID: {main_process.pid}")
        memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process, kill_thread))
        memory_thread.daemon = True  # Ensure the thread will exit when the main program exits
        memory_thread.start()
        
        with mp.Pool(processes=CPU_THREADS, maxtasksperchild=40) as pool:
            with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
                args = [(song_path, shared_song_results) for song_path in song_paths]
                for _ in pool.imap(process_song, args, chunksize=1):
                    pbar.update(1)

        kill_thread.value = True
        song_results = list(shared_song_results)

    # Aggregate results in the pandas dataframe
    songs_data_full = songs_data.copy(deep=True)
    for song_index, song_features in song_results:
        for feature, value in song_features.items():
            if feature not in songs_data_full.columns and isinstance(value, (tuple, set, list, np.ndarray, dict)):
                songs_data_full[feature] = np.nan
                songs_data_full[feature] = songs_data_full[feature].astype(object)
            songs_data_full.at[song_index, feature] = value

    return songs_data_full

In [ ]:
songs_data_full = process_songs()

Main Process ID: 767


Processing songs:   0%|          | 0/16391 [00:00<?, ?it/s]

{'Process 767': 520.609375, 'Child Process 1550': 397.06640625, 'Child Process 1560': 398.4375, 'Child Process 1563': 398.44921875, 'Child Process 1566': 398.45703125}
Top Consumer of Process 767: /tmp/ipykernel_767/2345274766.py:4: size=55.1 MiB, count=3, average=18.4 MiB
{'Process 767': 533.64453125, 'Child Process 1550': 398.25390625, 'Child Process 1560': 621.40625, 'Child Process 1563': 774.79296875, 'Child Process 1566': 688.02734375, 'Child Process 1569': 684.8515625}
Top Consumer of Process 767: /tmp/ipykernel_767/2345274766.py:4: size=55.1 MiB, count=3, average=18.4 MiB
{'Process 767': 534.64453125, 'Child Process 1550': 398.28125, 'Child Process 1560': 706.30078125, 'Child Process 1563': 657.45703125, 'Child Process 1566': 676.390625, 'Child Process 1569': 725.640625}
Top Consumer of Process 767: /tmp/ipykernel_767/2345274766.py:4: size=55.1 MiB, count=3, average=18.4 MiB
{'Process 767': 535.41796875, 'Child Process 1550': 398.83984375, 'Child Process 1560': 784.0234375, 'Chi

In [ ]:
songs_data_full.to_csv('data/songs_data_full_2.csv')

In [ ]:
songs_data_full.dropna(subset=['Danceability'])
